In [266]:
import pandas as pd
import numpy as np

In [366]:
train_label = pd.read_csv('../data/train_label.csv', index_col=0) 
val_label = pd.read_csv('../data/val_label.csv', index_col=0).reset_index()
val_label['subject_id'] = val_label['subject_id'].astype('str')
val_label['subject_id'] = 'User' + val_label['subject_id']

In [367]:
train_cols = pd.read_csv('../data/df_to_predict_val1.csv', index_col=0)
train_cols = train_cols.rename(columns={'userId':'subject_id'})
train_cols = train_cols[['subject_id','date','caffeine_y_caffeinated drink','alcohol_y_0','alcohol_y_wine','caffeine_y_coffee','caffeine_y_0', 'e4Eda_min','e4Temp_max','pmEmotion_y','pmStress_y']]

val_cols = pd.read_csv('../data/음성_컬럼수_줄인_val.csv', index_col=0)
val_cols = val_cols[['subject_id','date','Nature Sounds', 'Environmental Sounds', 'Human Activity Sounds', 'Machine and Traffic Sounds', 'Alert and Alarm Sounds', 'Music and Instrument Sounds', 'Other']]
val_cols['subject_id'] = val_cols['subject_id'].astype(str)
val_cols['subject_id'] = 'User' + val_cols['subject_id'] 

test_cols = pd.read_csv('../data/음성_컬럼수_줄인_test.csv', index_col=0)
test_cols = test_cols[['subject_id','date','Nature Sounds', 'Environmental Sounds', 'Human Activity Sounds', 'Machine and Traffic Sounds', 'Alert and Alarm Sounds', 'Music and Instrument Sounds', 'Other']]
test_cols['subject_id'] = test_cols['subject_id'].astype(str)
test_cols['subject_id'] = 'User' + test_cols['subject_id'] 

# gps 불포함 데이터
train 314 val 105 test 115

In [368]:
to_predict_val1 = pd.read_csv('../data/df_to_predict_val1.csv', index_col=0)
to_predict_test1 = pd.read_csv('../data/df_to_predict_test1.csv', index_col=0)
to_predict_train1 = pd.read_csv('../data/df_to_predict_train1.csv', index_col=0)

### val&test 컬럼 예측

In [369]:
from pycaret.datasets import get_data

In [370]:
categorical_pred_list = ['caffeine_y_caffeinated drink', 'alcohol_y_0', 'alcohol_y_wine', 'caffeine_y_coffee', 'caffeine_y_0', 'pmEmotion_y', 'pmStress_y']
numerical_pred_list = ['e4Eda_min', 'e4Temp_max']

##### val 예측

In [371]:
train = to_predict_val1.dropna(axis=0) #314행
test = to_predict_val1[to_predict_val1.isna().any(axis=1)] #105행

display(train.shape, test.shape)

(314, 29)

(105, 29)

In [372]:
from pycaret.classification import *
for i in range(0,len(categorical_pred_list)):
    train1 = train.copy()
    test1 = test.copy()
    
    y_train = train1[categorical_pred_list[i]]
    x_train = train1.drop(labels=categorical_pred_list, axis=1) 
    
    y_test = test1[categorical_pred_list[i]]
    x_test = test1.drop(labels=categorical_pred_list, axis=1) 
       
    clf = setup(data=pd.concat([x_train, y_train], axis=1), target=categorical_pred_list[i], train_size = 0.9, normalize = True)
    best_model = compare_models(sort='Accuracy')
    final_model = finalize_model(best_model)
    
    predictions = predict_model(final_model, data= x_test)
    test[categorical_pred_list[i]] = predictions.prediction_label

,Description,Value
0,Session id,1190
1,Target,caffeine_y_caffeinated drink
2,Target type,Binary
3,Original data shape,"(314, 23)"
4,Transformed data shape,"(314, 42)"
5,Transformed train set shape,"(282, 42)"
6,Transformed test set shape,"(32, 42)"
7,Numeric features,20
8,Categorical features,2
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,0.9786,0.0000,0.6000,0.5500,0.5667,0.5613,0.5657,0.2800
lr,Logistic Regression,0.9681,0.0000,0.1000,0.1000,0.1000,0.1000,0.1000,0.4260
svm,SVM - Linear Kernel,0.9681,0.0000,0.1000,0.1000,0.1000,0.1000,0.1000,0.0180
rf,Random Forest Classifier,0.9681,0.0000,0.1000,0.1000,0.1000,0.1000,0.1000,0.0380
ada,Ada Boost Classifier,0.9681,0.0000,0.1000,0.1000,0.1000,0.1000,0.1000,0.0310
et,Extra Trees Classifier,0.9681,0.0000,0.1000,0.1000,0.1000,0.1000,0.1000,0.0350
catboost,CatBoost Classifier,0.9681,0.0000,0.1000,0.1000,0.1000,0.1000,0.1000,0.8530
lda,Linear Discriminant Analysis,0.9680,0.0000,0.7000,0.5333,0.5833,0.5698,0.5870,0.0180
dt,Decision Tree Classifier,0.9645,0.0000,0.1000,0.1000,0.1000,0.0963,0.0963,0.0180
ridge,Ridge Classifier,0.9645,0.0000,0.4000,0.3000,0.3333,0.3226,0.3314,0.0250


,Description,Value
0,Session id,838
1,Target,alcohol_y_0
2,Target type,Binary
3,Original data shape,"(314, 23)"
4,Transformed data shape,"(314, 42)"
5,Transformed train set shape,"(282, 42)"
6,Transformed test set shape,"(32, 42)"
7,Numeric features,20
8,Categorical features,2
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dummy,Dummy Classifier,0.8369,0.0000,1.0000,0.8369,0.9111,0.0000,0.0000,0.0220
knn,K Neighbors Classifier,0.8367,0.0000,0.9663,0.8574,0.9081,0.1839,0.2292,0.0240
rf,Random Forest Classifier,0.8334,0.0000,0.9830,0.8441,0.9080,0.0529,0.0607,0.0450
catboost,CatBoost Classifier,0.8334,0.0000,0.9828,0.8441,0.9079,0.0503,0.0506,0.7600
et,Extra Trees Classifier,0.8264,0.0000,0.9616,0.8505,0.9024,0.1192,0.1349,0.0300
gbc,Gradient Boosting Classifier,0.8230,0.0000,0.9447,0.8584,0.8988,0.1608,0.1694,0.0440
lightgbm,Light Gradient Boosting Machine,0.8195,0.0000,0.9491,0.8520,0.8977,0.1204,0.1261,0.0720
ridge,Ridge Classifier,0.8188,0.0000,0.9486,0.8515,0.8971,0.1286,0.1479,0.0170
lda,Linear Discriminant Analysis,0.7974,0.0000,0.9016,0.8625,0.8808,0.1726,0.1880,0.0240
lr,Logistic Regression,0.7906,0.0000,0.9143,0.8466,0.8785,0.0860,0.1075,0.0190


,Description,Value
0,Session id,2679
1,Target,alcohol_y_wine
2,Target type,Binary
3,Original data shape,"(314, 23)"
4,Transformed data shape,"(314, 42)"
5,Transformed train set shape,"(282, 42)"
6,Transformed test set shape,"(32, 42)"
7,Numeric features,20
8,Categorical features,2
9,Preprocess,True


...

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.7915,0.0000,0.7626,0.8077,0.7807,0.5827,0.5880,0.0130
lr,Logistic Regression,0.7813,0.0000,0.7703,0.7870,0.7756,0.5626,0.5659,0.0140
knn,K Neighbors Classifier,0.7738,0.0000,0.7341,0.7968,0.7576,0.5470,0.5554,0.0160
et,Extra Trees Classifier,0.7735,0.0000,0.7484,0.7919,0.7636,0.5472,0.5549,0.0310
ridge,Ridge Classifier,0.7669,0.0000,0.7412,0.7796,0.7563,0.5333,0.5378,0.0130
svm,SVM - Linear Kernel,0.7417,0.0000,0.7626,0.7350,0.7426,0.4839,0.4927,0.0140
nb,Naive Bayes,0.7416,0.0000,0.5456,0.8993,0.6646,0.4789,0.5259,0.0130
qda,Quadratic Discriminant Analysis,0.7275,0.0000,0.6330,0.7913,0.6892,0.4523,0.4754,0.0130
gbc,Gradient Boosting Classifier,0.7132,0.0000,0.7275,0.7085,0.7115,0.4275,0.4350,0.0370
catboost,CatBoost Classifier,0.6957,0.0000,0.7346,0.6840,0.7023,0.3932,0.4004,0.8560


,Description,Value
0,Session id,2172
1,Target,pmEmotion_y
2,Target type,Multiclass
3,Target mapping,"1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4"
4,Original data shape,"(314, 23)"
5,Transformed data shape,"(314, 42)"
6,Transformed train set shape,"(282, 42)"
7,Transformed test set shape,"(32, 42)"
8,Numeric features,20
9,Categorical features,2


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.4613,0.0000,0.4613,0.4426,0.4369,0.1547,0.1579,0.0390
ridge,Ridge Classifier,0.4468,0.0000,0.4468,0.4275,0.4264,0.1361,0.1387,0.0130
rf,Random Forest Classifier,0.4467,0.0000,0.4467,0.4339,0.4212,0.1131,0.1156,0.0380
knn,K Neighbors Classifier,0.4432,0.0000,0.4432,0.4448,0.4289,0.1409,0.1436,0.0140
catboost,CatBoost Classifier,0.4398,0.0000,0.4398,0.4150,0.4099,0.1069,0.1105,3.0770
ada,Ada Boost Classifier,0.4363,0.0000,0.4363,0.4612,0.3805,0.1183,0.1486,0.0260
dummy,Dummy Classifier,0.4325,0.0000,0.4325,0.1871,0.2612,0.0000,0.0000,0.0180
lda,Linear Discriminant Analysis,0.4187,0.0000,0.4187,0.4562,0.4195,0.1318,0.1349,0.0170
gbc,Gradient Boosting Classifier,0.4115,0.0000,0.4115,0.4027,0.3956,0.0882,0.0905,0.1470
svm,SVM - Linear Kernel,0.4075,0.0000,0.4075,0.4082,0.4006,0.1120,0.1152,0.0160


,Description,Value
0,Session id,6208
1,Target,pmStress_y
2,Target type,Multiclass
3,Target mapping,"1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4"
4,Original data shape,"(314, 23)"
5,Transformed data shape,"(314, 42)"
6,Transformed train set shape,"(282, 42)"
7,Transformed test set shape,"(32, 42)"
8,Numeric features,20
9,Categorical features,2


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.4687,0.0000,0.4687,0.4321,0.4428,0.2652,0.2705,0.0140
catboost,CatBoost Classifier,0.4644,0.0000,0.4644,0.4320,0.4361,0.2531,0.2613,2.9480
lda,Linear Discriminant Analysis,0.4616,0.0000,0.4616,0.4700,0.4510,0.2738,0.2786,0.0260
rf,Random Forest Classifier,0.4612,0.0000,0.4612,0.4231,0.4236,0.2354,0.2507,0.0360
lr,Logistic Regression,0.4543,0.0000,0.4543,0.4618,0.4439,0.2611,0.2681,0.0140
et,Extra Trees Classifier,0.4506,0.0000,0.4506,0.4023,0.4168,0.2345,0.2414,0.0410
lightgbm,Light Gradient Boosting Machine,0.4502,0.0000,0.4502,0.4212,0.4202,0.2317,0.2421,0.3340
gbc,Gradient Boosting Classifier,0.4362,0.0000,0.4362,0.4199,0.4081,0.2200,0.2291,0.1420
knn,K Neighbors Classifier,0.4224,0.0000,0.4224,0.3747,0.3875,0.1932,0.1997,0.0130
svm,SVM - Linear Kernel,0.4011,0.0000,0.4011,0.4123,0.3921,0.1962,0.2001,0.0170


In [373]:
from pycaret.regression import *
for i in range(0,len(numerical_pred_list)):
    train1 = train.copy()
    test1 = test.copy()
    
    y_train = train1[numerical_pred_list[i]]
    x_train = train1.drop(labels=numerical_pred_list, axis=1) 
    
    y_test = test1[numerical_pred_list[i]]
    x_test = test1.drop(labels=numerical_pred_list, axis=1) 
       
    clf = setup(data=pd.concat([x_train, y_train], axis=1), target=numerical_pred_list[i], train_size = 0.9, normalize = True)
    best_model = compare_models()
    final_model = finalize_model(best_model)
    
    predictions = predict_model(final_model, data= x_test)
    test[numerical_pred_list[i]] = predictions.prediction_label

,Description,Value
0,Session id,6647
1,Target,e4Eda_min
2,Target type,Regression
3,Original data shape,"(314, 28)"
4,Transformed data shape,"(314, 47)"
5,Transformed train set shape,"(282, 47)"
6,Transformed test set shape,"(32, 47)"
7,Numeric features,25
8,Categorical features,2
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,0.0578,0.1485,0.2394,-0.1309,0.1239,7.1492,0.0170
knn,K Neighbors Regressor,0.0753,0.1554,0.2716,-15.1493,0.1472,22.7872,0.0180
et,Extra Trees Regressor,0.0856,0.1817,0.3070,-31.4289,0.1611,38.0781,0.0570
ada,AdaBoost Regressor,0.0833,0.1408,0.2562,-48.4630,0.1400,46.0970,0.0290
llar,Lasso Least Angle Regression,0.0931,0.1399,0.2487,-65.9586,0.1339,65.2195,0.0140
dummy,Dummy Regressor,0.0931,0.1399,0.2487,-65.9586,0.1339,65.2194,0.0170
en,Elastic Net,0.0931,0.1399,0.2487,-65.9586,0.1339,65.2195,0.0130
lasso,Lasso Regression,0.0931,0.1399,0.2487,-65.9586,0.1339,65.2195,0.0130
catboost,CatBoost Regressor,0.0885,0.1560,0.2860,-146.3553,0.1608,59.6866,0.7980
br,Bayesian Ridge,0.1031,0.1449,0.2681,-169.7314,0.1523,68.7486,0.0130


,Description,Value
0,Session id,5976
1,Target,e4Temp_max
2,Target type,Regression
3,Original data shape,"(314, 28)"
4,Transformed data shape,"(314, 47)"
5,Transformed train set shape,"(282, 47)"
6,Transformed test set shape,"(32, 47)"
7,Numeric features,25
8,Categorical features,2
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,0.3554,0.2525,0.4978,0.3188,0.0137,0.0100,0.0140
br,Bayesian Ridge,0.3828,0.2674,0.5123,0.2832,0.0141,0.0108,0.0120
et,Extra Trees Regressor,0.3823,0.2737,0.5191,0.2593,0.0143,0.0108,0.0590
gbr,Gradient Boosting Regressor,0.4013,0.2813,0.5254,0.2540,0.0144,0.0113,0.0350
huber,Huber Regressor,0.3842,0.2778,0.5216,0.2441,0.0143,0.0108,0.0150
ridge,Ridge Regression,0.3943,0.2856,0.5282,0.2262,0.0145,0.0111,0.0120
rf,Random Forest Regressor,0.4031,0.2915,0.5365,0.2195,0.0147,0.0114,0.0610
catboost,CatBoost Regressor,0.4029,0.2975,0.5405,0.2106,0.0149,0.0114,0.8220
lr,Linear Regression,0.4138,0.3105,0.5504,0.1594,0.0151,0.0117,0.0130
omp,Orthogonal Matching Pursuit,0.4489,0.3378,0.5784,0.0932,0.0159,0.0126,0.0130


In [374]:
test = test.rename(columns={'userId':'subject_id'})
test = pd.merge(test, val_cols, on=['subject_id','date'])
test = pd.merge(test, val_label, on=['subject_id','date'])
test

,date,month,weekday,weekend,subject_id,activity,IN_VEHICLE,ON_BICYCLE,STILL,UNKNOWN,...,Alert and Alarm Sounds,Music and Instrument Sounds,Other,Q1,Q2,Q3,S1,S2,S3,S4
0,2023-08-20,8,0,1,User1,1440,105,17,1153,0,...,2,113,45,1,1,1,0,0,0,0
1,2023-08-21,8,1,0,User1,1440,92,25,1061,0,...,3,61,48,1,1,1,0,0,1,0
2,2023-08-22,8,1,0,User1,1440,105,48,869,0,...,1,132,61,0,1,1,0,1,1,0
3,2023-08-23,8,1,0,User1,1440,133,51,963,0,...,5,128,51,0,1,1,0,0,1,0
4,2023-08-24,8,1,0,User1,1440,141,10,1069,0,...,2,100,45,1,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,2023-10-27,10,1,0,User4,1431,73,0,1336,1,...,1,27,45,0,1,0,0,1,1,1
101,2023-10-28,10,0,1,User4,1440,47,0,1385,0,...,2,122,48,1,1,0,1,1,1,1
102,2023-10-29,10,0,1,User4,1440,121,0,1303,0,...,3,37,52,1,1,0,0,1,1,1
103,2023-10-30,10,1,0,User4,1440,60,0,1369,0,...,2,28,65,0,1,0,0,0,1,1


In [375]:
test.to_csv('../data/val_final_v1.csv')

##### test 예측

In [376]:
train = to_predict_test1.dropna(axis=0) #314행
test = to_predict_test1[to_predict_test1.isna().any(axis=1)] #105행

display(train.shape, test.shape)

(314, 29)

(115, 29)

In [377]:
from pycaret.classification import *
for i in range(0,len(categorical_pred_list)):
    train1 = train.copy()
    test1 = test.copy()
    
    y_train = train1[categorical_pred_list[i]]
    x_train = train1.drop(labels=categorical_pred_list, axis=1) 
    
    y_test = test1[categorical_pred_list[i]]
    x_test = test1.drop(labels=categorical_pred_list, axis=1) 
       
    clf = setup(data=pd.concat([x_train, y_train], axis=1), target=categorical_pred_list[i], train_size = 0.9, normalize = True)
    best_model = compare_models(sort='Accuracy')
    final_model = finalize_model(best_model)
    
    predictions = predict_model(final_model, data= x_test)
    test[categorical_pred_list[i]] = predictions.prediction_label

,Description,Value
0,Session id,8343
1,Target,caffeine_y_caffeinated drink
2,Target type,Binary
3,Original data shape,"(314, 23)"
4,Transformed data shape,"(314, 42)"
5,Transformed train set shape,"(282, 42)"
6,Transformed test set shape,"(32, 42)"
7,Numeric features,20
8,Categorical features,2
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,0.9714,0.0000,0.5000,0.4500,0.4667,0.4576,0.4620,0.0150
lda,Linear Discriminant Analysis,0.9714,0.0000,0.7000,0.5500,0.6000,0.5876,0.6008,0.0170
svm,SVM - Linear Kernel,0.9680,0.0000,0.2000,0.2000,0.2000,0.1963,0.1963,0.0140
ridge,Ridge Classifier,0.9680,0.0000,0.6000,0.4500,0.5000,0.4876,0.5008,0.0130
rf,Random Forest Classifier,0.9680,0.0000,0.1000,0.1000,0.1000,0.1000,0.1000,0.0360
et,Extra Trees Classifier,0.9680,0.0000,0.1000,0.1000,0.1000,0.1000,0.1000,0.0380
catboost,CatBoost Classifier,0.9680,0.0000,0.1000,0.1000,0.1000,0.1000,0.1000,0.9360
qda,Quadratic Discriminant Analysis,0.9645,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0140
ada,Ada Boost Classifier,0.9645,0.0000,0.1000,0.1000,0.1000,0.0964,0.0964,0.0250
lightgbm,Light Gradient Boosting Machine,0.9645,0.0000,0.1000,0.1000,0.1000,0.0964,0.0964,0.0900


,Description,Value
0,Session id,4180
1,Target,alcohol_y_0
2,Target type,Binary
3,Original data shape,"(314, 23)"
4,Transformed data shape,"(314, 42)"
5,Transformed train set shape,"(282, 42)"
6,Transformed test set shape,"(32, 42)"
7,Numeric features,20
8,Categorical features,2
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,0.8403,0.0000,0.9663,0.8605,0.9100,0.1956,0.2358,0.0180
rf,Random Forest Classifier,0.8369,0.0000,0.9958,0.8395,0.9108,0.0231,0.0334,0.0340
catboost,CatBoost Classifier,0.8369,0.0000,0.9915,0.8420,0.9105,0.0531,0.0716,0.9180
dummy,Dummy Classifier,0.8369,0.0000,1.0000,0.8369,0.9111,0.0000,0.0000,0.0190
et,Extra Trees Classifier,0.8366,0.0000,0.9705,0.8546,0.9087,0.1431,0.1695,0.0370
ridge,Ridge Classifier,0.8225,0.0000,0.9576,0.8497,0.9002,0.1014,0.1250,0.0120
gbc,Gradient Boosting Classifier,0.8224,0.0000,0.9659,0.8444,0.9008,0.0634,0.0841,0.0380
lr,Logistic Regression,0.8012,0.0000,0.9279,0.8492,0.8863,0.0791,0.0921,0.0200
ada,Ada Boost Classifier,0.8011,0.0000,0.9279,0.8492,0.8861,0.0849,0.1079,0.0250
lightgbm,Light Gradient Boosting Machine,0.7908,0.0000,0.9279,0.8399,0.8803,0.0062,-0.0039,0.0920


,Description,Value
0,Session id,2727
1,Target,alcohol_y_wine
2,Target type,Binary
3,Original data shape,"(314, 23)"
4,Transformed data shape,"(314, 42)"
5,Transformed train set shape,"(282, 42)"
6,Transformed test set shape,"(32, 42)"
7,Numeric features,20
8,Categorical features,2
9,Preprocess,True


...

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,0.7773,0.0000,0.7698,0.7877,0.7698,0.5546,0.5683,0.0170
et,Extra Trees Classifier,0.7665,0.0000,0.7555,0.7811,0.7583,0.5334,0.5466,0.0410
ridge,Ridge Classifier,0.7592,0.0000,0.7396,0.7696,0.7487,0.5183,0.5250,0.0130
nb,Naive Bayes,0.7528,0.0000,0.6753,0.7975,0.7215,0.5029,0.5170,0.0150
lda,Linear Discriminant Analysis,0.7488,0.0000,0.7396,0.7568,0.7415,0.4977,0.5055,0.0190
lr,Logistic Regression,0.7452,0.0000,0.7473,0.7490,0.7438,0.4915,0.4965,0.0170
qda,Quadratic Discriminant Analysis,0.7276,0.0000,0.6676,0.7680,0.7061,0.4553,0.4684,0.0200
svm,SVM - Linear Kernel,0.6991,0.0000,0.6846,0.7181,0.6852,0.3991,0.4158,0.0190
rf,Random Forest Classifier,0.6989,0.0000,0.7341,0.6966,0.7061,0.4001,0.4110,0.0500
gbc,Gradient Boosting Classifier,0.6953,0.0000,0.7049,0.6955,0.6901,0.3922,0.4044,0.0500


,Description,Value
0,Session id,7559
1,Target,pmEmotion_y
2,Target type,Multiclass
3,Target mapping,"1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4"
4,Original data shape,"(314, 23)"
5,Transformed data shape,"(314, 42)"
6,Transformed train set shape,"(282, 42)"
7,Transformed test set shape,"(32, 42)"
8,Numeric features,20
9,Categorical features,2


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.5177,0.0000,0.5177,0.4781,0.4876,0.2444,0.2531,0.0460
ridge,Ridge Classifier,0.5005,0.0000,0.5005,0.4685,0.4785,0.2241,0.2274,0.0130
lda,Linear Discriminant Analysis,0.4645,0.0000,0.4645,0.4716,0.4557,0.1969,0.2014,0.0220
rf,Random Forest Classifier,0.4573,0.0000,0.4573,0.4142,0.4205,0.1279,0.1377,0.0470
knn,K Neighbors Classifier,0.4403,0.0000,0.4403,0.4402,0.4246,0.1411,0.1452,0.0140
gbc,Gradient Boosting Classifier,0.4397,0.0000,0.4397,0.4207,0.4170,0.1213,0.1247,0.1830
catboost,CatBoost Classifier,0.4394,0.0000,0.4394,0.4013,0.4046,0.1057,0.1106,3.2130
dummy,Dummy Classifier,0.4328,0.0000,0.4328,0.1876,0.2616,0.0000,0.0000,0.0200
svm,SVM - Linear Kernel,0.4151,0.0000,0.4151,0.4211,0.4065,0.1242,0.1263,0.0150
lr,Logistic Regression,0.4116,0.0000,0.4116,0.3954,0.3942,0.1191,0.1221,0.0180


,Description,Value
0,Session id,4659
1,Target,pmStress_y
2,Target type,Multiclass
3,Target mapping,"1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4"
4,Original data shape,"(314, 23)"
5,Transformed data shape,"(314, 42)"
6,Transformed train set shape,"(282, 42)"
7,Transformed test set shape,"(32, 42)"
8,Numeric features,20
9,Categorical features,2


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.4683,0.0000,0.4683,0.4219,0.4378,0.2512,0.2565,3.2420
et,Extra Trees Classifier,0.4435,0.0000,0.4435,0.4160,0.4183,0.2305,0.2357,0.0540
ridge,Ridge Classifier,0.4399,0.0000,0.4399,0.4031,0.4160,0.2273,0.2308,0.0150
rf,Random Forest Classifier,0.4398,0.0000,0.4398,0.3795,0.3991,0.2009,0.2090,0.0450
lr,Logistic Regression,0.4113,0.0000,0.4113,0.4055,0.3967,0.1912,0.1939,0.0200
lda,Linear Discriminant Analysis,0.4078,0.0000,0.4078,0.4116,0.4029,0.2070,0.2098,0.0190
lightgbm,Light Gradient Boosting Machine,0.4011,0.0000,0.4011,0.3735,0.3748,0.1644,0.1680,0.4040
knn,K Neighbors Classifier,0.3977,0.0000,0.3977,0.3758,0.3759,0.1702,0.1746,0.0140
gbc,Gradient Boosting Classifier,0.3977,0.0000,0.3977,0.3670,0.3735,0.1588,0.1638,0.1900
qda,Quadratic Discriminant Analysis,0.3616,0.0000,0.3616,0.3108,0.3071,0.1351,0.1475,0.0150


In [378]:
from pycaret.regression import *
for i in range(0,len(numerical_pred_list)):
    train1 = train.copy()
    test1 = test.copy()
    
    y_train = train1[numerical_pred_list[i]]
    x_train = train1.drop(labels=numerical_pred_list, axis=1) 
    
    y_test = test1[numerical_pred_list[i]]
    x_test = test1.drop(labels=numerical_pred_list, axis=1) 
       
    clf = setup(data=pd.concat([x_train, y_train], axis=1), target=numerical_pred_list[i], train_size = 0.9, normalize = True)
    best_model = compare_models()
    final_model = finalize_model(best_model)
    
    predictions = predict_model(final_model, data= x_test)
    test[numerical_pred_list[i]] = predictions.prediction_label

,Description,Value
0,Session id,6417
1,Target,e4Eda_min
2,Target type,Regression
3,Original data shape,"(314, 28)"
4,Transformed data shape,"(314, 47)"
5,Transformed train set shape,"(282, 47)"
6,Transformed test set shape,"(32, 47)"
7,Numeric features,25
8,Categorical features,2
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,0.0538,0.1425,0.2498,-0.4950,0.1312,3.5493,0.0200
llar,Lasso Least Angle Regression,0.0941,0.1400,0.2517,-4.2025,0.1357,62.3695,0.0260
lasso,Lasso Regression,0.0941,0.1400,0.2517,-4.2025,0.1357,62.3695,0.0170
dummy,Dummy Regressor,0.0941,0.1400,0.2517,-4.2025,0.1357,62.3695,0.0150
en,Elastic Net,0.0941,0.1400,0.2517,-4.2025,0.1357,62.3695,0.0160
knn,K Neighbors Regressor,0.0718,0.1529,0.2839,-5.5071,0.1560,19.4292,0.0210
dt,Decision Tree Regressor,0.0815,0.2394,0.3527,-5.9835,0.1704,15.8270,0.0170
br,Bayesian Ridge,0.0993,0.1422,0.2604,-6.2848,0.1451,62.8138,0.0210
catboost,CatBoost Regressor,0.0800,0.1441,0.2557,-9.5445,0.1400,48.4724,0.9080
et,Extra Trees Regressor,0.0839,0.1816,0.3133,-10.3228,0.1628,29.9850,0.0570


,Description,Value
0,Session id,6036
1,Target,e4Temp_max
2,Target type,Regression
3,Original data shape,"(314, 28)"
4,Transformed data shape,"(314, 47)"
5,Transformed train set shape,"(282, 47)"
6,Transformed test set shape,"(32, 47)"
7,Numeric features,25
8,Categorical features,2
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,0.3584,0.2962,0.5091,0.3495,0.0139,0.0101,0.0220
et,Extra Trees Regressor,0.3671,0.2817,0.5120,0.3235,0.0140,0.0103,0.0610
br,Bayesian Ridge,0.3826,0.3046,0.5287,0.2917,0.0144,0.0108,0.0160
huber,Huber Regressor,0.3939,0.3114,0.5356,0.2622,0.0146,0.0111,0.0260
ridge,Ridge Regression,0.3957,0.3122,0.5380,0.2549,0.0147,0.0111,0.0170
rf,Random Forest Regressor,0.4061,0.3425,0.5589,0.2368,0.0153,0.0114,0.0810
gbr,Gradient Boosting Regressor,0.4071,0.3344,0.5570,0.2132,0.0152,0.0114,0.0460
catboost,CatBoost Regressor,0.4078,0.3604,0.5730,0.2039,0.0156,0.0115,0.8710
omp,Orthogonal Matching Pursuit,0.4367,0.3850,0.5936,0.1290,0.0161,0.0123,0.0170
lasso,Lasso Regression,0.4913,0.4562,0.6508,-0.0158,0.0178,0.0138,0.0120


In [379]:
test = test.rename(columns={'userId':'subject_id'})
test = pd.merge(test, test_cols, on=['subject_id','date'])
test

,date,month,weekday,weekend,subject_id,activity,IN_VEHICLE,ON_BICYCLE,STILL,UNKNOWN,...,mAcc_max,mAcc_min,mAcc_std,Nature Sounds,Environmental Sounds,Human Activity Sounds,Machine and Traffic Sounds,Alert and Alarm Sounds,Music and Instrument Sounds,Other
0,2023-11-05,11,0,1,User5,1434,122,3,843,396,...,95.964407,0.023213,0.698846,160,106,276,37,8,64,52
1,2023-11-06,11,1,0,User5,1440,98,0,1279,0,...,76.107229,0.053643,0.749389,200,108,205,66,4,64,73
2,2023-11-07,11,1,0,User5,1440,64,0,1345,0,...,126.778495,0.029615,0.731316,197,96,278,35,9,35,70
3,2023-11-08,11,1,0,User5,1435,19,0,540,845,...,88.220726,0.031125,0.578272,180,108,240,51,0,91,46
4,2023-11-09,11,1,0,User5,1440,48,0,1379,0,...,100.005972,0.053160,0.514795,196,73,267,69,4,43,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,2023-11-05,11,0,1,User8,1440,18,0,1418,0,...,106.572897,0.244744,0.489330,8,461,157,26,1,33,34
111,2023-11-06,11,1,0,User8,1432,86,0,1130,1,...,103.682283,0.131424,1.203481,33,374,190,55,2,37,13
112,2023-11-07,11,1,0,User8,1431,108,0,1288,0,...,92.053778,0.575628,0.681906,21,463,159,26,0,17,28
113,2023-11-08,11,1,0,User8,1437,9,0,1218,69,...,87.060951,0.303318,0.983404,10,401,263,10,0,12,19


In [380]:
test.to_csv('../data/test_final_v1.csv')

### train 컬럼 예측

In [381]:
pred_list = ['Nature Sounds', 'Environmental Sounds', 'Human Activity Sounds',
       'Machine and Traffic Sounds', 'Alert and Alarm Sounds',
       'Music and Instrument Sounds', 'Other']

In [382]:
train = to_predict_train1.dropna(axis=0) #220행 = 105+115
test = to_predict_train1[to_predict_train1.isna().any(axis=1)] #314행

display(train.shape, test.shape)

(220, 27)

(314, 27)

In [383]:
from pycaret.regression import *
for i in range(0,len(pred_list)):
    train1 = train.copy()
    test1 = test.copy()
    
    y_train = train1[pred_list[i]]
    x_train = train1.drop(labels=pred_list, axis=1) 
    
    y_test = test1[pred_list[i]]
    x_test = test1.drop(labels=pred_list, axis=1) 
       
    clf = setup(data=pd.concat([x_train, y_train], axis=1).reset_index(drop=True), target=pred_list[i], train_size = 0.9, normalize = True)
    best_model = compare_models()
    final_model = finalize_model(best_model)
    
    predictions = predict_model(final_model, data= x_test)
    test[pred_list[i]] = predictions.prediction_label

,Description,Value
0,Session id,814
1,Target,Nature Sounds
2,Target type,Regression
3,Original data shape,"(220, 21)"
4,Transformed data shape,"(220, 28)"
5,Transformed train set shape,"(198, 28)"
6,Transformed test set shape,"(22, 28)"
7,Numeric features,18
8,Categorical features,2
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,32.0611,2345.6760,47.7119,0.2745,1.9703,4.8326,0.0150
catboost,CatBoost Regressor,34.0779,2647.1089,50.4084,0.2033,2.0409,3.6563,0.6670
ada,AdaBoost Regressor,36.0882,2750.9100,51.3268,0.1715,2.2030,4.4690,0.0290
rf,Random Forest Regressor,35.8431,2958.9919,53.2107,0.1115,2.0660,4.1916,0.0530
gbr,Gradient Boosting Regressor,35.8133,2993.9361,53.1108,0.1004,2.0225,4.2871,0.0320
et,Extra Trees Regressor,35.4062,2958.0145,53.3329,0.0859,2.0571,3.6951,0.0400
en,Elastic Net,42.3069,3009.5677,54.4966,0.0759,2.3712,5.3434,0.0120
lightgbm,Light Gradient Boosting Machine,41.0646,3281.5635,56.6391,-0.0081,2.2960,5.0043,0.0640
dummy,Dummy Regressor,44.8491,3476.0106,58.5371,-0.0328,2.5318,5.2853,0.0150
br,Bayesian Ridge,44.3571,3506.8596,58.3548,-0.0932,2.3598,5.6446,0.0170


,Description,Value
0,Session id,4511
1,Target,Environmental Sounds
2,Target type,Regression
3,Original data shape,"(220, 21)"
4,Transformed data shape,"(220, 28)"
5,Transformed train set shape,"(198, 28)"
6,Transformed test set shape,"(22, 28)"
7,Numeric features,18
8,Categorical features,2
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,159.7636,43265.8100,203.0565,0.1402,0.5749,0.6297,0.0130
en,Elastic Net,178.7027,43060.9066,205.8276,0.1323,0.5799,0.6426,0.0130
lr,Linear Regression,182.2834,46017.7107,212.9265,0.0631,0.6380,0.6176,0.0120
ridge,Ridge Regression,182.5838,46123.4269,213.1988,0.0612,0.6350,0.6191,0.0130
br,Bayesian Ridge,182.8899,46317.4850,213.7093,0.0585,0.6498,0.6234,0.0120
huber,Huber Regressor,183.3932,46799.0288,214.5129,0.0479,0.6438,0.6252,0.0160
lasso,Lasso Regression,184.2843,46880.1470,214.9097,0.0473,0.6511,0.6300,0.0130
llar,Lasso Least Angle Regression,185.4406,47463.2174,216.1576,0.0358,0.6528,0.6363,0.0130
par,Passive Aggressive Regressor,188.0232,49514.5809,220.6685,-0.0103,0.6619,0.6504,0.0250
dummy,Dummy Regressor,208.7152,52431.9746,228.4724,-0.0783,0.6407,0.7887,0.0150


,Description,Value
0,Session id,6576
1,Target,Human Activity Sounds
2,Target type,Regression
3,Original data shape,"(220, 21)"
4,Transformed data shape,"(220, 28)"
5,Transformed train set shape,"(198, 28)"
6,Transformed test set shape,"(22, 28)"
7,Numeric features,18
8,Categorical features,2
9,Preprocess,True


...

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,1.1499,3.2169,1.7083,-0.1202,0.6620,0.5919,0.0180
dummy,Dummy Regressor,1.3541,3.2984,1.7477,-0.2268,0.7112,0.4630,0.0180
llar,Lasso Least Angle Regression,1.3634,3.3612,1.7689,-0.2733,0.7180,0.4790,0.0150
lasso,Lasso Regression,1.3634,3.3612,1.7689,-0.2733,0.7180,0.4790,0.0120
en,Elastic Net,1.3825,3.5429,1.8224,-0.3846,0.7350,0.5268,0.0140
catboost,CatBoost Regressor,1.3398,3.8705,1.9099,-0.5357,0.7416,0.6940,0.6550
ada,AdaBoost Regressor,1.4899,4.0087,1.9394,-0.5648,0.7709,0.5714,0.0280
huber,Huber Regressor,1.4225,4.1223,1.9741,-0.7275,0.7440,0.7543,0.0180
br,Bayesian Ridge,1.5006,4.1866,2.0021,-0.8429,0.7632,0.7447,0.0120
et,Extra Trees Regressor,1.4162,4.4441,2.0459,-0.9098,0.8013,0.7693,0.0380


,Description,Value
0,Session id,3821
1,Target,Music and Instrument Sounds
2,Target type,Regression
3,Original data shape,"(220, 21)"
4,Transformed data shape,"(220, 28)"
5,Transformed train set shape,"(198, 28)"
6,Transformed test set shape,"(22, 28)"
7,Numeric features,18
8,Categorical features,2
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,23.0032,1184.7134,33.5056,0.0359,2.0252,1.6848,0.0120
en,Elastic Net,25.8105,1257.5532,34.8478,-0.0289,2.1418,2.7412,0.0120
dummy,Dummy Regressor,27.7918,1295.4454,35.4421,-0.0568,2.2841,2.4950,0.0150
catboost,CatBoost Regressor,24.9134,1337.6829,35.7796,-0.0918,2.0477,2.4577,0.6920
et,Extra Trees Regressor,24.9746,1414.0161,36.8070,-0.1577,2.0741,1.7639,0.0490
ada,AdaBoost Regressor,27.6044,1418.1921,37.0418,-0.1591,2.1902,2.9098,0.0270
br,Bayesian Ridge,27.4961,1396.7752,36.8225,-0.1612,2.1540,3.0631,0.0130
ridge,Ridge Regression,27.6376,1418.3246,37.1460,-0.1838,2.1449,3.0885,0.0110
lr,Linear Regression,27.6339,1420.4153,37.1823,-0.1866,2.1403,3.0861,0.0140
huber,Huber Regressor,27.7759,1497.4020,38.0684,-0.2397,2.1360,3.3204,0.0150


,Description,Value
0,Session id,2022
1,Target,Other
2,Target type,Regression
3,Original data shape,"(220, 21)"
4,Transformed data shape,"(220, 28)"
5,Transformed train set shape,"(198, 28)"
6,Transformed test set shape,"(22, 28)"
7,Numeric features,18
8,Categorical features,2
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
en,Elastic Net,22.8108,784.1284,27.1783,0.0134,2.0465,1.3876,0.0120
knn,K Neighbors Regressor,21.3581,916.2989,28.9709,-0.1041,1.9489,1.0197,0.0140
dummy,Dummy Regressor,25.4298,936.5139,29.5450,-0.1352,2.2237,2.4695,0.0170
br,Bayesian Ridge,24.4128,914.9168,29.2226,-0.1624,2.0979,1.5400,0.0120
ada,AdaBoost Regressor,24.3214,933.1893,29.7488,-0.1757,2.0984,1.4492,0.0270
ridge,Ridge Regression,24.6612,941.5025,29.5854,-0.1940,2.0892,1.4977,0.0110
lr,Linear Regression,24.6722,944.4752,29.6187,-0.1969,2.0785,1.4850,0.0140
llar,Lasso Least Angle Regression,25.0772,944.9378,29.8906,-0.2142,2.1018,1.6310,0.0120
lasso,Lasso Regression,25.0777,944.9832,29.8912,-0.2143,2.1018,1.6311,0.0120
catboost,CatBoost Regressor,23.4936,955.1719,30.1439,-0.2161,2.0074,1.2354,0.6660


In [384]:
test = test.rename(columns={'userId':'subject_id'})
test = pd.merge(test, train_cols, on=['subject_id','date'])
test = pd.merge(test, train_label, on=['subject_id','date'])
test

,date,month,weekday,weekend,subject_id,activity,IN_VEHICLE,ON_BICYCLE,STILL,UNKNOWN,...,e4Temp_max,pmEmotion_y,pmStress_y,Q1,Q2,Q3,S1,S2,S3,S4
0,2020-08-31,8,1,0,user01,779,101,0,523,127,...,35.07,4.0,3.0,0,0,0,0,1,1,1
1,2020-09-05,9,0,1,user01,698,40,0,530,77,...,35.11,4.0,4.0,1,0,1,1,1,0,1
2,2020-09-07,9,1,0,user01,731,41,0,589,54,...,35.50,4.0,4.0,1,0,1,1,1,1,1
3,2020-09-12,9,0,1,user01,414,15,0,323,59,...,35.25,4.0,4.0,0,0,1,0,0,1,1
4,2020-09-13,9,0,1,user01,658,84,0,489,52,...,35.39,4.0,4.0,0,0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,2020-09-21,9,1,0,user30,836,5,0,762,32,...,35.49,3.0,3.0,0,0,1,0,1,0,0
310,2020-09-22,9,1,0,user30,757,6,0,650,59,...,35.16,3.0,3.0,0,0,1,0,1,1,1
311,2020-09-23,9,1,0,user30,841,9,1,733,54,...,35.61,3.0,3.0,0,0,1,0,1,1,0
312,2020-09-24,9,1,0,user30,755,4,1,663,47,...,35.31,3.0,3.0,1,0,1,0,1,1,1


In [385]:
test.to_csv('../data/train_final_v1.csv')

# gps 포함 데이터

In [402]:
to_predict_val2 = pd.read_csv('../data/df_to_predict_val2.csv', index_col=0)
to_predict_test2 = pd.read_csv('../data/df_to_predict_test2.csv', index_col=0)
to_predict_train2 = pd.read_csv('../data/df_to_predict_train2.csv', index_col=0)

### val & test 컬럼 예측

##### val 예측

In [408]:
train = to_predict_val2.dropna(axis=0) #288행
test = to_predict_val2[to_predict_val2.isna().any(axis=1)] #101행

display(train.shape, test.shape)

(288, 32)

(101, 32)

In [409]:
to_predict_val2

,date,month,weekday,weekend,subject_id,activity,IN_VEHICLE,ON_BICYCLE,STILL,UNKNOWN,WALKING,RUNNING,caffeine_y_caffeinated drink,alcohol_y_0,alcohol_y_wine,caffeine_y_coffee,caffeine_y_0,e4Eda_min,e4Temp_max,pmEmotion_y,pmStress_y,Hr_average,Hr_max,Hr_min,Hr_std,mAcc_average,mAcc_max,mAcc_min,mAcc_std,gps_distance,max_distance,distance_ratio
0,2020-08-31,8,1,0,user01,779,101,0,523,127,0,0,0.0,1.0,0.0,1.0,0.0,0.016916,35.07,4.0,3.0,88.048321,118.98,61.63,8.985455,9.890882,111.525495,0.457250,1.357032,93.417040,104.800651,0.891378
1,2020-09-05,9,0,1,user01,698,40,0,530,77,0,0,0.0,0.0,0.0,1.0,0.0,0.001281,35.11,4.0,4.0,86.948462,160.87,57.43,13.987130,9.920032,68.971885,0.057910,1.564459,28.073488,104.800651,0.267875
2,2020-09-07,9,1,0,user01,731,41,0,589,54,0,0,0.0,1.0,0.0,1.0,0.0,0.001281,35.50,4.0,4.0,83.276502,165.08,54.07,12.863235,9.898526,77.797261,0.424273,1.277668,24.984973,104.800651,0.238405
3,2020-09-12,9,0,1,user01,414,15,0,323,59,0,0,0.0,1.0,0.0,0.0,1.0,0.001281,35.25,4.0,4.0,85.026952,133.08,55.53,13.203510,9.971321,85.082452,0.448292,1.219717,20.536303,104.800651,0.195956
4,2020-09-13,9,0,1,user01,658,84,0,489,52,0,0,0.0,1.0,0.0,0.0,1.0,0.001280,35.39,4.0,4.0,85.962022,143.28,55.02,13.591047,9.925301,80.154310,0.297016,1.277832,104.800651,104.800651,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,2023-10-27,10,1,0,User4,1431,73,0,1336,1,21,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.934150,137.00,62.00,12.875535,9.798069,80.680862,0.282069,0.398572,21.067014,225.699928,0.093341
97,2023-10-28,10,0,1,User4,1440,47,0,1385,0,8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.346939,135.00,60.00,11.835901,9.842360,70.815633,0.236974,0.310147,14.722594,225.699928,0.065231
98,2023-10-29,10,0,1,User4,1440,121,0,1303,0,16,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.784861,145.00,56.00,12.171161,9.841872,84.311013,0.598077,0.518713,31.145407,225.699928,0.137995
99,2023-10-30,10,1,0,User4,1440,60,0,1369,0,11,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.525661,148.00,59.00,11.992490,9.793543,68.829471,0.608869,0.342009,35.936848,225.699928,0.159224


In [410]:
from pycaret.classification import *
for i in range(0,len(categorical_pred_list)):
    train1 = train.copy()
    test1 = test.copy()
    
    y_train = train1[categorical_pred_list[i]]
    x_train = train1.drop(labels=categorical_pred_list, axis=1) 
    
    y_test = test1[categorical_pred_list[i]]
    x_test = test1.drop(labels=categorical_pred_list, axis=1) 
       
    clf = setup(data=pd.concat([x_train, y_train], axis=1), target=categorical_pred_list[i], train_size = 0.9, normalize = True)
    best_model = compare_models(sort='Accuracy')
    final_model = finalize_model(best_model)
    
    predictions = predict_model(final_model, data= x_test)
    test[categorical_pred_list[i]] = predictions.prediction_label

,Description,Value
0,Session id,6872
1,Target,caffeine_y_caffeinated drink
2,Target type,Binary
3,Original data shape,"(288, 26)"
4,Transformed data shape,"(288, 45)"
5,Transformed train set shape,"(259, 45)"
6,Transformed test set shape,"(29, 45)"
7,Numeric features,23
8,Categorical features,2
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,0.9731,0.0000,0.6000,0.6000,0.6000,0.5906,0.5902,0.2850
et,Extra Trees Classifier,0.9692,0.0000,0.2000,0.2000,0.2000,0.2000,0.2000,0.0440
rf,Random Forest Classifier,0.9654,0.0000,0.1000,0.1000,0.1000,0.1000,0.1000,0.0650
lr,Logistic Regression,0.9615,0.0000,0.1000,0.1000,0.1000,0.0960,0.0960,0.4200
svm,SVM - Linear Kernel,0.9615,0.0000,0.1000,0.1000,0.1000,0.0960,0.0960,0.0190
lda,Linear Discriminant Analysis,0.9615,0.0000,0.7000,0.5833,0.6167,0.6010,0.6131,0.0190
lightgbm,Light Gradient Boosting Machine,0.9615,0.0000,0.1000,0.1000,0.1000,0.0960,0.0960,0.0730
qda,Quadratic Discriminant Analysis,0.9614,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0230
dummy,Dummy Classifier,0.9614,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0200
dt,Decision Tree Classifier,0.9577,0.0000,0.1000,0.1000,0.1000,0.0920,0.0920,0.0170


,Description,Value
0,Session id,938
1,Target,alcohol_y_0
2,Target type,Binary
3,Original data shape,"(288, 26)"
4,Transformed data shape,"(288, 45)"
5,Transformed train set shape,"(259, 45)"
6,Transformed test set shape,"(29, 45)"
7,Numeric features,23
8,Categorical features,2
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dummy,Dummy Classifier,0.8378,0.0000,1.0000,0.8378,0.9117,0.0000,0.0000,0.0160
catboost,CatBoost Classifier,0.8338,0.0000,0.9907,0.8399,0.9089,0.0227,0.0295,0.8180
rf,Random Forest Classifier,0.8300,0.0000,0.9861,0.8393,0.9066,0.0161,0.0209,0.0570
et,Extra Trees Classifier,0.8300,0.0000,0.9587,0.8573,0.9044,0.1557,0.1654,0.0310
knn,K Neighbors Classifier,0.8262,0.0000,0.9448,0.8628,0.9012,0.1877,0.1865,0.0250
ridge,Ridge Classifier,0.8223,0.0000,0.9582,0.8495,0.9000,0.1035,0.1151,0.0190
lr,Logistic Regression,0.8029,0.0000,0.9305,0.8496,0.8875,0.0874,0.0910,0.0200
lda,Linear Discriminant Analysis,0.8028,0.0000,0.9167,0.8589,0.8857,0.1412,0.1422,0.0150
gbc,Gradient Boosting Classifier,0.7954,0.0000,0.9359,0.8395,0.8842,0.0123,0.0115,0.0390
lightgbm,Light Gradient Boosting Machine,0.7762,0.0000,0.9171,0.8328,0.8723,-0.0461,-0.0555,0.0610


,Description,Value
0,Session id,2494
1,Target,alcohol_y_wine
2,Target type,Binary
3,Original data shape,"(288, 26)"
4,Transformed data shape,"(288, 45)"
5,Transformed train set shape,"(259, 45)"
6,Transformed test set shape,"(29, 45)"
7,Numeric features,23
8,Categorical features,2
9,Preprocess,True


...

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.7798,0.0000,0.7912,0.8039,0.7916,0.5579,0.5664,0.0330
knn,K Neighbors Classifier,0.7486,0.0000,0.7253,0.7907,0.7546,0.4972,0.5011,0.0130
lda,Linear Discriminant Analysis,0.7294,0.0000,0.7544,0.7520,0.7449,0.4556,0.4682,0.0140
svm,SVM - Linear Kernel,0.7260,0.0000,0.7709,0.7524,0.7492,0.4465,0.4636,0.0190
ridge,Ridge Classifier,0.7255,0.0000,0.7544,0.7467,0.7428,0.4475,0.4597,0.0140
lr,Logistic Regression,0.7218,0.0000,0.7401,0.7485,0.7367,0.4409,0.4527,0.0140
nb,Naive Bayes,0.7218,0.0000,0.6396,0.8346,0.7113,0.4465,0.4693,0.0160
qda,Quadratic Discriminant Analysis,0.6951,0.0000,0.7473,0.7309,0.7118,0.3835,0.4070,0.0140
rf,Random Forest Classifier,0.6912,0.0000,0.7055,0.7285,0.7024,0.3809,0.3948,0.0360
catboost,CatBoost Classifier,0.6834,0.0000,0.6978,0.7065,0.6950,0.3657,0.3759,0.9120


,Description,Value
0,Session id,5415
1,Target,pmEmotion_y
2,Target type,Multiclass
3,Target mapping,"1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4"
4,Original data shape,"(288, 26)"
5,Transformed data shape,"(288, 45)"
6,Transformed train set shape,"(259, 45)"
7,Transformed test set shape,"(29, 45)"
8,Numeric features,23
9,Categorical features,2


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.5366,0.0000,0.5366,0.5116,0.5062,0.2815,0.2902,0.0420
ridge,Ridge Classifier,0.5097,0.0000,0.5097,0.4876,0.4834,0.2427,0.2489,0.0140
rf,Random Forest Classifier,0.5060,0.0000,0.5060,0.4867,0.4714,0.2203,0.2346,0.0390
knn,K Neighbors Classifier,0.4945,0.0000,0.4945,0.4873,0.4759,0.2300,0.2368,0.0140
lr,Logistic Regression,0.4940,0.0000,0.4940,0.4802,0.4767,0.2359,0.2396,0.0190
svm,SVM - Linear Kernel,0.4868,0.0000,0.4868,0.4803,0.4661,0.2386,0.2448,0.0160
catboost,CatBoost Classifier,0.4788,0.0000,0.4788,0.4383,0.4367,0.1775,0.1889,3.0730
lda,Linear Discriminant Analysis,0.4745,0.0000,0.4745,0.4651,0.4603,0.2095,0.2143,0.0160
gbc,Gradient Boosting Classifier,0.4595,0.0000,0.4595,0.4438,0.4406,0.1781,0.1822,0.1440
dt,Decision Tree Classifier,0.4438,0.0000,0.4438,0.4517,0.4411,0.1883,0.1912,0.0150


,Description,Value
0,Session id,3667
1,Target,pmStress_y
2,Target type,Multiclass
3,Target mapping,"1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4"
4,Original data shape,"(288, 26)"
5,Transformed data shape,"(288, 45)"
6,Transformed train set shape,"(259, 45)"
7,Transformed test set shape,"(29, 45)"
8,Numeric features,23
9,Categorical features,2


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.4632,0.0000,0.4632,0.4250,0.4319,0.2548,0.2622,3.1930
rf,Random Forest Classifier,0.4514,0.0000,0.4514,0.4059,0.4136,0.2279,0.2372,0.0370
lightgbm,Light Gradient Boosting Machine,0.4398,0.0000,0.4398,0.4042,0.4127,0.2318,0.2366,0.2910
et,Extra Trees Classifier,0.4394,0.0000,0.4394,0.3952,0.4073,0.2211,0.2262,0.0440
ridge,Ridge Classifier,0.4357,0.0000,0.4357,0.3997,0.4057,0.2236,0.2303,0.0190
gbc,Gradient Boosting Classifier,0.4051,0.0000,0.4051,0.3722,0.3751,0.1777,0.1837,0.1570
lr,Logistic Regression,0.4009,0.0000,0.4009,0.3688,0.3805,0.1868,0.1886,0.0210
lda,Linear Discriminant Analysis,0.3894,0.0000,0.3894,0.3898,0.3797,0.1857,0.1886,0.0170
knn,K Neighbors Classifier,0.3858,0.0000,0.3858,0.3654,0.3664,0.1593,0.1652,0.0160
dummy,Dummy Classifier,0.3552,0.0000,0.3552,0.1264,0.1864,0.0000,0.0000,0.0160


In [411]:
from pycaret.regression import *
for i in range(0,len(numerical_pred_list)):
    train1 = train.copy()
    test1 = test.copy()
    
    y_train = train1[numerical_pred_list[i]]
    x_train = train1.drop(labels=numerical_pred_list, axis=1) 
    
    y_test = test1[numerical_pred_list[i]]
    x_test = test1.drop(labels=numerical_pred_list, axis=1) 
       
    clf = setup(data=pd.concat([x_train, y_train], axis=1), target=numerical_pred_list[i], train_size = 0.9, normalize = True)
    best_model = compare_models()
    final_model = finalize_model(best_model)
    
    predictions = predict_model(final_model, data= x_test)
    test[numerical_pred_list[i]] = predictions.prediction_label

,Description,Value
0,Session id,637
1,Target,e4Eda_min
2,Target type,Regression
3,Original data shape,"(288, 31)"
4,Transformed data shape,"(288, 50)"
5,Transformed train set shape,"(259, 50)"
6,Transformed test set shape,"(29, 50)"
7,Numeric features,28
8,Categorical features,2
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,0.0375,0.0556,0.1429,-32.2594,0.0886,9.1331,0.0130
llar,Lasso Least Angle Regression,0.0546,0.0531,0.1364,-51.8589,0.0854,38.2026,0.0260
lasso,Lasso Regression,0.0546,0.0531,0.1364,-51.8589,0.0854,38.2026,0.0130
dummy,Dummy Regressor,0.0546,0.0531,0.1364,-51.8589,0.0854,38.2026,0.0180
en,Elastic Net,0.0546,0.0531,0.1364,-51.8589,0.0854,38.2026,0.0140
ada,AdaBoost Regressor,0.0501,0.0541,0.1399,-62.1111,0.0886,28.4610,0.0280
catboost,CatBoost Regressor,0.0487,0.0572,0.1642,-184.7770,0.1066,33.5250,0.9190
lightgbm,Light Gradient Boosting Machine,0.0687,0.0614,0.1817,-413.0759,0.1174,51.9934,0.0750
huber,Huber Regressor,0.0377,0.0612,0.1508,-481.2295,0.0914,7.4398,0.0170
br,Bayesian Ridge,0.0759,0.0590,0.1750,-547.9205,0.1181,61.0554,0.0130


,Description,Value
0,Session id,5767
1,Target,e4Temp_max
2,Target type,Regression
3,Original data shape,"(288, 31)"
4,Transformed data shape,"(288, 50)"
5,Transformed train set shape,"(259, 50)"
6,Transformed test set shape,"(29, 50)"
7,Numeric features,28
8,Categorical features,2
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,0.3678,0.2844,0.5019,0.2947,0.0137,0.0103,0.0190
br,Bayesian Ridge,0.3802,0.2880,0.5131,0.2849,0.0140,0.0107,0.0130
et,Extra Trees Regressor,0.3794,0.2783,0.5055,0.2827,0.0138,0.0106,0.0510
rf,Random Forest Regressor,0.3937,0.3056,0.5272,0.2826,0.0144,0.0111,0.0660
huber,Huber Regressor,0.3978,0.2984,0.5237,0.2390,0.0143,0.0112,0.0180
ridge,Ridge Regression,0.4018,0.3054,0.5298,0.2156,0.0144,0.0113,0.0130
catboost,CatBoost Regressor,0.4129,0.3340,0.5516,0.2069,0.0150,0.0116,0.9430
gbr,Gradient Boosting Regressor,0.4143,0.3249,0.5479,0.1935,0.0149,0.0116,0.0390
lr,Linear Regression,0.4128,0.3174,0.5400,0.1866,0.0147,0.0116,0.0130
ada,AdaBoost Regressor,0.4444,0.3901,0.6001,0.0632,0.0164,0.0125,0.0330


In [412]:
display(test.head(),val_cols.head(),val_label.head())

,date,month,weekday,weekend,subject_id,activity,IN_VEHICLE,ON_BICYCLE,STILL,UNKNOWN,...,Hr_max,Hr_min,Hr_std,mAcc_average,mAcc_max,mAcc_min,mAcc_std,gps_distance,max_distance,distance_ratio
0,2023-08-20,8,0,1,User1,1440,105,17,1153,0,...,126.0,47.0,11.811378,9.892758,168.708665,0.136139,0.898369,12.728039,118.121587,0.107754
1,2023-08-21,8,1,0,User1,1440,92,25,1061,0,...,147.0,48.0,12.433426,9.992590,174.162067,0.054774,1.576221,10.879238,118.121587,0.092102
2,2023-08-22,8,1,0,User1,1440,105,48,869,0,...,120.0,37.0,11.689185,9.893863,161.544672,0.127270,0.956041,11.372360,118.121587,0.096277
3,2023-08-23,8,1,0,User1,1440,133,51,963,0,...,113.0,32.0,13.355582,9.932777,148.546601,0.023076,1.052149,18.676591,118.121587,0.158113
4,2023-08-24,8,1,0,User1,1440,141,10,1069,0,...,109.0,50.0,10.740390,9.897462,181.592017,0.058856,0.969007,7.164520,118.121587,0.060654


,subject_id,date,Nature Sounds,Environmental Sounds,Human Activity Sounds,Machine and Traffic Sounds,Alert and Alarm Sounds,Music and Instrument Sounds,Other
0,User1,2023-08-20,17,167,309,67,2,113,45
1,User1,2023-08-21,20,278,259,51,3,61,48
2,User1,2023-08-22,26,189,247,64,1,132,61
3,User1,2023-08-23,38,210,209,79,5,128,51
4,User1,2023-08-24,21,206,270,76,2,100,45


,subject_id,date,Q1,Q2,Q3,S1,S2,S3,S4
0,User1,2023-08-20,1,1,1,0,0,0,0
1,User1,2023-08-21,1,1,1,0,0,1,0
2,User1,2023-08-22,0,1,1,0,1,1,0
3,User1,2023-08-23,0,1,1,0,0,1,0
4,User1,2023-08-24,1,1,1,0,0,1,0


In [413]:
test = test.rename(columns={'userId':'subject_id'})
test = pd.merge(test, val_cols, on=['subject_id','date'])
test = pd.merge(test, val_label, on=['subject_id','date'])
test

,date,month,weekday,weekend,subject_id,activity,IN_VEHICLE,ON_BICYCLE,STILL,UNKNOWN,...,Alert and Alarm Sounds,Music and Instrument Sounds,Other,Q1,Q2,Q3,S1,S2,S3,S4
0,2023-08-20,8,0,1,User1,1440,105,17,1153,0,...,2,113,45,1,1,1,0,0,0,0
1,2023-08-21,8,1,0,User1,1440,92,25,1061,0,...,3,61,48,1,1,1,0,0,1,0
2,2023-08-22,8,1,0,User1,1440,105,48,869,0,...,1,132,61,0,1,1,0,1,1,0
3,2023-08-23,8,1,0,User1,1440,133,51,963,0,...,5,128,51,0,1,1,0,0,1,0
4,2023-08-24,8,1,0,User1,1440,141,10,1069,0,...,2,100,45,1,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,2023-10-27,10,1,0,User4,1431,73,0,1336,1,...,1,27,45,0,1,0,0,1,1,1
97,2023-10-28,10,0,1,User4,1440,47,0,1385,0,...,2,122,48,1,1,0,1,1,1,1
98,2023-10-29,10,0,1,User4,1440,121,0,1303,0,...,3,37,52,1,1,0,0,1,1,1
99,2023-10-30,10,1,0,User4,1440,60,0,1369,0,...,2,28,65,0,1,0,0,0,1,1


In [414]:
test.to_csv('../data/val_final_v2.csv')

##### test 예측

In [415]:
train = to_predict_test2.dropna(axis=0) #288행
test = to_predict_test2[to_predict_test2.isna().any(axis=1)] #113행

display(train.shape, test.shape)

(288, 32)

(115, 32)

In [416]:
from pycaret.classification import *
for i in range(0,len(categorical_pred_list)):
    train1 = train.copy()
    test1 = test.copy()
    
    y_train = train1[categorical_pred_list[i]]
    x_train = train1.drop(labels=categorical_pred_list, axis=1) 
    
    y_test = test1[categorical_pred_list[i]]
    x_test = test1.drop(labels=categorical_pred_list, axis=1) 
       
    clf = setup(data=pd.concat([x_train, y_train], axis=1), target=categorical_pred_list[i], train_size = 0.9, normalize = True)
    best_model = compare_models(sort='Accuracy')
    final_model = finalize_model(best_model)
    
    predictions = predict_model(final_model, data= x_test)
    test[categorical_pred_list[i]] = predictions.prediction_label

,Description,Value
0,Session id,8395
1,Target,caffeine_y_caffeinated drink
2,Target type,Binary
3,Original data shape,"(288, 26)"
4,Transformed data shape,"(288, 45)"
5,Transformed train set shape,"(259, 45)"
6,Transformed test set shape,"(29, 45)"
7,Numeric features,23
8,Categorical features,2
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,0.9692,0.0000,0.6000,0.4333,0.4833,0.4767,0.4939,0.0150
svm,SVM - Linear Kernel,0.9691,0.0000,0.2000,0.2000,0.2000,0.2000,0.2000,0.0250
catboost,CatBoost Classifier,0.9652,0.0000,0.2000,0.1500,0.1667,0.1649,0.1693,1.0140
rf,Random Forest Classifier,0.9614,0.0000,0.1000,0.1000,0.1000,0.0960,0.0960,0.0360
qda,Quadratic Discriminant Analysis,0.9614,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0140
et,Extra Trees Classifier,0.9614,0.0000,0.1000,0.1000,0.1000,0.0960,0.0960,0.0420
dummy,Dummy Classifier,0.9614,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0210
lda,Linear Discriminant Analysis,0.9577,0.0000,0.7000,0.5167,0.5667,0.5493,0.5703,0.0180
lr,Logistic Regression,0.9575,0.0000,0.1000,0.0500,0.0667,0.0609,0.0653,0.0160
ada,Ada Boost Classifier,0.9575,0.0000,0.1000,0.0500,0.0667,0.0609,0.0653,0.0290


,Description,Value
0,Session id,8962
1,Target,alcohol_y_0
2,Target type,Binary
3,Original data shape,"(288, 26)"
4,Transformed data shape,"(288, 45)"
5,Transformed train set shape,"(259, 45)"
6,Transformed test set shape,"(29, 45)"
7,Numeric features,23
8,Categorical features,2
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.8380,0.0000,0.9909,0.8435,0.9111,0.0588,0.0766,0.0350
dummy,Dummy Classifier,0.8378,0.0000,1.0000,0.8378,0.9117,0.0000,0.0000,0.0180
catboost,CatBoost Classifier,0.8302,0.0000,0.9864,0.8395,0.9067,0.0126,0.0106,1.0230
et,Extra Trees Classifier,0.8263,0.0000,0.9589,0.8529,0.9024,0.1275,0.1533,0.0460
knn,K Neighbors Classifier,0.8225,0.0000,0.9403,0.8606,0.8983,0.2000,0.2318,0.0140
ridge,Ridge Classifier,0.8189,0.0000,0.9407,0.8576,0.8965,0.1748,0.2035,0.0120
gbc,Gradient Boosting Classifier,0.8148,0.0000,0.9496,0.8484,0.8954,0.0819,0.0861,0.0470
qda,Quadratic Discriminant Analysis,0.8071,0.0000,0.9078,0.8706,0.8863,0.1902,0.2061,0.0150
lda,Linear Discriminant Analysis,0.8032,0.0000,0.9126,0.8618,0.8851,0.1782,0.2006,0.0230
lightgbm,Light Gradient Boosting Machine,0.7914,0.0000,0.9312,0.8378,0.8812,0.0152,0.0271,0.0790


,Description,Value
0,Session id,7128
1,Target,alcohol_y_wine
2,Target type,Binary
3,Original data shape,"(288, 26)"
4,Transformed data shape,"(288, 45)"
5,Transformed train set shape,"(259, 45)"
6,Transformed test set shape,"(29, 45)"
7,Numeric features,23
8,Categorical features,2
9,Preprocess,True


...

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,0.7645,0.0000,0.7484,0.8014,0.7711,0.5293,0.5343,0.0140
ridge,Ridge Classifier,0.7454,0.0000,0.7407,0.7811,0.7570,0.4891,0.4942,0.0130
lda,Linear Discriminant Analysis,0.7454,0.0000,0.7407,0.7812,0.7566,0.4892,0.4948,0.0210
lr,Logistic Regression,0.7338,0.0000,0.7555,0.7577,0.7526,0.4638,0.4705,0.0140
et,Extra Trees Classifier,0.7295,0.0000,0.7335,0.7630,0.7432,0.4572,0.4633,0.0430
qda,Quadratic Discriminant Analysis,0.7180,0.0000,0.8214,0.7117,0.7508,0.4270,0.4624,0.0160
catboost,CatBoost Classifier,0.7143,0.0000,0.7555,0.7284,0.7388,0.4232,0.4265,1.1120
svm,SVM - Linear Kernel,0.7138,0.0000,0.7335,0.7398,0.7333,0.4245,0.4286,0.0140
rf,Random Forest Classifier,0.7105,0.0000,0.7549,0.7228,0.7368,0.4147,0.4172,0.0380
nb,Naive Bayes,0.6986,0.0000,0.8418,0.6849,0.7492,0.3816,0.4084,0.0140


,Description,Value
0,Session id,6286
1,Target,pmEmotion_y
2,Target type,Multiclass
3,Target mapping,"1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4"
4,Original data shape,"(288, 26)"
5,Transformed data shape,"(288, 45)"
6,Transformed train set shape,"(259, 45)"
7,Transformed test set shape,"(29, 45)"
8,Numeric features,23
9,Categorical features,2


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,0.5171,0.0000,0.5171,0.5248,0.4963,0.2628,0.2755,0.0180
ada,Ada Boost Classifier,0.5060,0.0000,0.5060,0.4333,0.4306,0.2001,0.2563,0.0280
et,Extra Trees Classifier,0.5054,0.0000,0.5054,0.4967,0.4786,0.2410,0.2504,0.0500
rf,Random Forest Classifier,0.4943,0.0000,0.4943,0.4639,0.4574,0.2047,0.2223,0.0450
ridge,Ridge Classifier,0.4902,0.0000,0.4902,0.4825,0.4708,0.2176,0.2240,0.0170
gbc,Gradient Boosting Classifier,0.4789,0.0000,0.4789,0.4668,0.4575,0.2037,0.2107,0.1910
lda,Linear Discriminant Analysis,0.4748,0.0000,0.4748,0.5057,0.4689,0.2194,0.2265,0.0210
catboost,CatBoost Classifier,0.4671,0.0000,0.4671,0.4183,0.4289,0.1611,0.1703,3.5470
lightgbm,Light Gradient Boosting Machine,0.4635,0.0000,0.4635,0.4260,0.4372,0.1776,0.1818,0.3960
svm,SVM - Linear Kernel,0.4555,0.0000,0.4555,0.4662,0.4467,0.1909,0.1946,0.0200


,Description,Value
0,Session id,6012
1,Target,pmStress_y
2,Target type,Multiclass
3,Target mapping,"1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4"
4,Original data shape,"(288, 26)"
5,Transformed data shape,"(288, 45)"
6,Transformed train set shape,"(259, 45)"
7,Transformed test set shape,"(29, 45)"
8,Numeric features,23
9,Categorical features,2


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.4942,0.0000,0.4942,0.4493,0.4505,0.3011,0.3157,0.0570
rf,Random Forest Classifier,0.4748,0.0000,0.4748,0.4248,0.4344,0.2572,0.2705,0.0430
gbc,Gradient Boosting Classifier,0.4711,0.0000,0.4711,0.4493,0.4511,0.2705,0.2772,0.2090
catboost,CatBoost Classifier,0.4477,0.0000,0.4477,0.4045,0.4101,0.2326,0.2443,3.4850
lightgbm,Light Gradient Boosting Machine,0.4325,0.0000,0.4325,0.4059,0.4064,0.2134,0.2209,0.3300
lr,Logistic Regression,0.4288,0.0000,0.4288,0.4344,0.4122,0.2312,0.2381,0.0140
ridge,Ridge Classifier,0.4288,0.0000,0.4288,0.4108,0.3972,0.2133,0.2252,0.0150
knn,K Neighbors Classifier,0.4168,0.0000,0.4168,0.3935,0.3880,0.1916,0.2018,0.0170
lda,Linear Discriminant Analysis,0.4135,0.0000,0.4135,0.4211,0.3943,0.2151,0.2227,0.0220
qda,Quadratic Discriminant Analysis,0.4017,0.0000,0.4017,0.3573,0.3663,0.1661,0.1744,0.0180


In [417]:
from pycaret.regression import *
for i in range(0,len(numerical_pred_list)):
    train1 = train.copy()
    test1 = test.copy()
    
    y_train = train1[numerical_pred_list[i]]
    x_train = train1.drop(labels=numerical_pred_list, axis=1) 
    
    y_test = test1[numerical_pred_list[i]]
    x_test = test1.drop(labels=numerical_pred_list, axis=1) 
       
    clf = setup(data=pd.concat([x_train, y_train], axis=1), target=numerical_pred_list[i], train_size = 0.9, normalize = True)
    best_model = compare_models()
    final_model = finalize_model(best_model)
    
    predictions = predict_model(final_model, data= x_test)
    test[numerical_pred_list[i]] = predictions.prediction_label

,Description,Value
0,Session id,6885
1,Target,e4Eda_min
2,Target type,Regression
3,Original data shape,"(288, 31)"
4,Transformed data shape,"(288, 50)"
5,Transformed train set shape,"(259, 50)"
6,Transformed test set shape,"(29, 50)"
7,Numeric features,28
8,Categorical features,2
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,0.0593,0.1578,0.2498,-2.1817,0.1287,4.1828,0.0200
knn,K Neighbors Regressor,0.0671,0.1586,0.2641,-20.5536,0.1409,15.9768,0.0240
llar,Lasso Least Angle Regression,0.1027,0.1543,0.2613,-23.6338,0.1412,73.6157,0.0140
dummy,Dummy Regressor,0.1027,0.1543,0.2613,-23.6338,0.1412,73.6157,0.0180
en,Elastic Net,0.1027,0.1543,0.2613,-23.6338,0.1412,73.6157,0.0140
lasso,Lasso Regression,0.1027,0.1543,0.2613,-23.6338,0.1412,73.6157,0.0130
br,Bayesian Ridge,0.1105,0.1556,0.2780,-58.0916,0.1558,80.3863,0.0160
catboost,CatBoost Regressor,0.0967,0.1667,0.2930,-59.6888,0.1637,57.1523,0.9940
et,Extra Trees Regressor,0.0972,0.1930,0.3310,-124.0936,0.1797,44.7877,0.0550
lightgbm,Light Gradient Boosting Machine,0.1467,0.1866,0.3446,-153.8907,0.2011,105.3536,0.0710


,Description,Value
0,Session id,8852
1,Target,e4Temp_max
2,Target type,Regression
3,Original data shape,"(288, 31)"
4,Transformed data shape,"(288, 50)"
5,Transformed train set shape,"(259, 50)"
6,Transformed test set shape,"(29, 50)"
7,Numeric features,28
8,Categorical features,2
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,0.3550,0.2883,0.5152,0.3222,0.0141,0.0100,0.0260
br,Bayesian Ridge,0.3907,0.3235,0.5464,0.2353,0.0149,0.0110,0.0260
rf,Random Forest Regressor,0.4020,0.3466,0.5648,0.2146,0.0154,0.0113,0.0780
catboost,CatBoost Regressor,0.4094,0.3583,0.5720,0.1875,0.0156,0.0115,0.9550
et,Extra Trees Regressor,0.3951,0.3354,0.5638,0.1852,0.0154,0.0111,0.0610
huber,Huber Regressor,0.3986,0.3402,0.5565,0.1727,0.0151,0.0112,0.0190
gbr,Gradient Boosting Regressor,0.4109,0.3504,0.5725,0.1640,0.0156,0.0116,0.0600
ridge,Ridge Regression,0.4125,0.3469,0.5721,0.1117,0.0156,0.0116,0.0120
lr,Linear Regression,0.4203,0.3566,0.5796,0.0898,0.0158,0.0118,0.0140
ada,AdaBoost Regressor,0.4632,0.4311,0.6324,0.0156,0.0173,0.0131,0.0320


In [418]:
test.isnull().sum()

date                            0
month                           0
weekday                         0
weekend                         0
subject_id                      0
activity                        0
IN_VEHICLE                      0
ON_BICYCLE                      0
STILL                           0
UNKNOWN                         0
WALKING                         0
RUNNING                         0
caffeine_y_caffeinated drink    0
alcohol_y_0                     0
alcohol_y_wine                  0
caffeine_y_coffee               0
caffeine_y_0                    0
e4Eda_min                       0
e4Temp_max                      0
pmEmotion_y                     0
pmStress_y                      0
Hr_average                      0
Hr_max                          0
Hr_min                          0
Hr_std                          0
mAcc_average                    0
mAcc_max                        0
mAcc_min                        0
mAcc_std                        0
gps_distance  

In [419]:
test = test.rename(columns={'userId':'subject_id'})
test = pd.merge(test, test_cols, on=['subject_id','date'])
test

,date,month,weekday,weekend,subject_id,activity,IN_VEHICLE,ON_BICYCLE,STILL,UNKNOWN,...,gps_distance,max_distance,distance_ratio,Nature Sounds,Environmental Sounds,Human Activity Sounds,Machine and Traffic Sounds,Alert and Alarm Sounds,Music and Instrument Sounds,Other
0,2023-11-05,11,0,1,User5,1434,122,3,843,396,...,19.189911,23.507009,0.816349,160,106,276,37,8,64,52
1,2023-11-06,11,1,0,User5,1440,98,0,1279,0,...,23.507009,23.507009,1.000000,200,108,205,66,4,64,73
2,2023-11-07,11,1,0,User5,1440,64,0,1345,0,...,19.830692,23.507009,0.843608,197,96,278,35,9,35,70
3,2023-11-08,11,1,0,User5,1435,19,0,540,845,...,18.805130,23.507009,0.799980,180,108,240,51,0,91,46
4,2023-11-09,11,1,0,User5,1440,48,0,1379,0,...,21.914918,23.507009,0.932272,196,73,267,69,4,43,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,2023-11-05,11,0,1,User8,1440,18,0,1418,0,...,6.616197,338.004687,0.019574,8,461,157,26,1,33,34
111,2023-11-06,11,1,0,User8,1432,86,0,1130,1,...,58.828139,338.004687,0.174045,33,374,190,55,2,37,13
112,2023-11-07,11,1,0,User8,1431,108,0,1288,0,...,20.485850,338.004687,0.060608,21,463,159,26,0,17,28
113,2023-11-08,11,1,0,User8,1437,9,0,1218,69,...,13.449348,338.004687,0.039790,10,401,263,10,0,12,19


In [420]:
test.to_csv('../data/test_final_v2.csv')

### train 컬럼 예측

In [428]:
train = to_predict_train2.dropna(axis=0) #214행
test = to_predict_train2[to_predict_train2.isna().any(axis=1)] #288행

display(train.shape, test.shape)

(214, 30)

(290, 30)

In [430]:
from pycaret.regression import *
for i in range(0,len(pred_list)):
    train1 = train.copy()
    test1 = test.copy()
    
    y_train = train1[pred_list[i]].reset_index(drop=True)
    x_train = train1.drop(labels=pred_list, axis=1) .reset_index(drop=True)
    
    y_test = test1[pred_list[i]].reset_index(drop=True)
    x_test = test1.drop(labels=pred_list, axis=1).reset_index(drop=True) 
       
    clf = setup(data=pd.concat([x_train, y_train], axis=1).reset_index(drop=True), target=pred_list[i], train_size = 0.9, normalize = True)
    best_model = compare_models()
    final_model = finalize_model(best_model)
    
    predictions = predict_model(final_model, data= x_test)
    test[pred_list[i]] = predictions.prediction_label

,Description,Value
0,Session id,1289
1,Target,Nature Sounds
2,Target type,Regression
3,Original data shape,"(214, 24)"
4,Transformed data shape,"(214, 31)"
5,Transformed train set shape,"(192, 31)"
6,Transformed test set shape,"(22, 31)"
7,Numeric features,21
8,Categorical features,2
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,24.6772,1911.7947,40.8849,0.4993,1.0525,1.5783,0.0490
gbr,Gradient Boosting Regressor,26.0812,2054.3673,42.3781,0.4529,1.2600,2.0591,0.0280
catboost,CatBoost Regressor,26.8028,2079.8533,42.9673,0.4443,1.4078,2.0478,0.4960
ada,AdaBoost Regressor,30.1343,2212.2209,43.4931,0.4312,1.8790,2.8534,0.0220
lightgbm,Light Gradient Boosting Machine,28.8471,2275.1251,44.9211,0.3907,1.5488,2.4155,0.0540
knn,K Neighbors Regressor,29.5052,2049.5446,43.7044,0.3873,1.7494,3.1519,0.0120
et,Extra Trees Regressor,32.1101,2575.8708,48.5123,0.2742,1.7507,2.9967,0.0320
dt,Decision Tree Regressor,27.1345,2744.2045,50.2174,0.2196,1.0531,1.7465,0.0120
en,Elastic Net,43.1514,3157.9753,54.9380,0.0719,2.3636,5.4237,0.0140
br,Bayesian Ridge,45.6753,3479.6600,57.5227,-0.0427,2.3624,5.6892,0.0140


,Description,Value
0,Session id,7687
1,Target,Environmental Sounds
2,Target type,Regression
3,Original data shape,"(214, 24)"
4,Transformed data shape,"(214, 31)"
5,Transformed train set shape,"(192, 31)"
6,Transformed test set shape,"(22, 31)"
7,Numeric features,21
8,Categorical features,2
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,55.5039,8446.3932,82.5029,0.8345,0.2568,0.2087,0.0480
gbr,Gradient Boosting Regressor,69.2808,11616.2169,98.6198,0.7616,0.2947,0.2474,0.0310
lightgbm,Light Gradient Boosting Machine,85.9633,12907.9838,109.9663,0.7365,0.3253,0.2891,0.0560
catboost,CatBoost Regressor,87.7358,13410.3101,112.0308,0.7227,0.3547,0.3296,0.5500
ada,AdaBoost Regressor,84.9870,14988.7059,117.3586,0.6849,0.3446,0.3001,0.0270
dt,Decision Tree Regressor,75.9163,18582.9453,124.1241,0.5842,0.3530,0.2650,0.0120
et,Extra Trees Regressor,134.7662,34573.1684,180.8715,0.2670,0.4823,0.4864,0.0430
knn,K Neighbors Regressor,135.0242,33656.9841,179.8960,0.2527,0.4823,0.4812,0.0130
en,Elastic Net,164.9531,37814.1068,191.9305,0.1788,0.5481,0.5998,0.0130
ridge,Ridge Regression,165.6084,39821.2011,197.0972,0.1296,0.5389,0.5640,0.0120


,Description,Value
0,Session id,5582
1,Target,Human Activity Sounds
2,Target type,Regression
3,Original data shape,"(214, 24)"
4,Transformed data shape,"(214, 31)"
5,Transformed train set shape,"(192, 31)"
6,Transformed test set shape,"(22, 31)"
7,Numeric features,21
8,Categorical features,2
9,Preprocess,True


...

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.9736,2.6169,1.5617,0.1345,0.5342,0.5982,0.5430
gbr,Gradient Boosting Regressor,0.9785,2.7571,1.6005,0.0943,0.5376,0.6623,0.0270
knn,K Neighbors Regressor,1.0466,2.7495,1.6065,0.0828,0.5897,0.6035,0.0120
rf,Random Forest Regressor,0.9515,2.7379,1.6032,0.0756,0.5299,0.6402,0.0460
ada,AdaBoost Regressor,1.1395,2.8506,1.6233,0.0719,0.5993,0.5752,0.0270
lightgbm,Light Gradient Boosting Machine,1.1060,3.1068,1.7024,-0.0539,0.5900,0.6853,0.0570
dummy,Dummy Regressor,1.3740,3.3053,1.7595,-0.0542,0.7127,0.4780,0.0160
lasso,Lasso Regression,1.3849,3.3612,1.7783,-0.0867,0.7189,0.5117,0.0120
llar,Lasso Least Angle Regression,1.3849,3.3612,1.7783,-0.0867,0.7189,0.5117,0.0150
dt,Decision Tree Regressor,1.0116,3.3426,1.7592,-0.0882,0.6176,0.8445,0.0140


,Description,Value
0,Session id,4907
1,Target,Music and Instrument Sounds
2,Target type,Regression
3,Original data shape,"(214, 24)"
4,Transformed data shape,"(214, 31)"
5,Transformed train set shape,"(192, 31)"
6,Transformed test set shape,"(22, 31)"
7,Numeric features,21
8,Categorical features,2
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,21.4697,1045.6539,31.1421,0.1463,1.7640,1.4176,0.0130
catboost,CatBoost Regressor,19.7886,1078.9228,31.4438,0.1255,1.4491,1.7727,0.6390
lightgbm,Light Gradient Boosting Machine,20.4744,1167.8149,32.9213,-0.0420,1.4463,1.9913,0.0560
rf,Random Forest Regressor,19.2497,1198.7628,33.1631,-0.0677,1.2124,1.8060,0.0440
ada,AdaBoost Regressor,20.9678,1138.4579,32.4035,-0.0739,1.6050,2.2153,0.0250
dummy,Dummy Regressor,29.3526,1391.5976,36.4292,-0.1467,2.2637,2.2434,0.0170
en,Elastic Net,25.2864,1263.3068,34.6228,-0.1553,2.0153,2.1158,0.0170
et,Extra Trees Regressor,21.8492,1280.6872,34.2294,-0.2169,1.6363,1.8289,0.0410
gbr,Gradient Boosting Regressor,20.7078,1287.6529,34.4731,-0.2466,1.4623,1.8741,0.0330
br,Bayesian Ridge,26.9156,1467.6549,37.3375,-0.4311,1.9893,2.3991,0.0180


,Description,Value
0,Session id,2039
1,Target,Other
2,Target type,Regression
3,Original data shape,"(214, 24)"
4,Transformed data shape,"(214, 31)"
5,Transformed train set shape,"(192, 31)"
6,Transformed test set shape,"(22, 31)"
7,Numeric features,21
8,Categorical features,2
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,9.8514,278.9652,15.9773,0.6894,0.7993,0.4146,0.0570
gbr,Gradient Boosting Regressor,11.0463,305.0136,16.7116,0.6665,0.9905,0.5483,0.0330
catboost,CatBoost Regressor,12.4167,332.0567,17.6237,0.6347,1.2731,0.8006,0.6370
dt,Decision Tree Regressor,11.6518,417.1492,19.3355,0.5359,0.8776,0.5320,0.0150
lightgbm,Light Gradient Boosting Machine,14.6856,441.2485,20.4334,0.5118,1.4381,0.9519,0.0610
ada,AdaBoost Regressor,14.4020,436.6664,20.5073,0.5035,1.4156,0.8509,0.0330
et,Extra Trees Regressor,16.7328,596.5294,24.0098,0.2995,1.4939,1.0770,0.0410
knn,K Neighbors Regressor,19.0271,723.7087,26.6166,0.1711,1.7384,1.0257,0.0140
en,Elastic Net,22.4488,790.5278,27.8977,0.0913,1.9537,1.3647,0.0140
dummy,Dummy Regressor,25.5680,961.2162,30.5743,-0.0436,2.1951,2.4297,0.0160


In [434]:
test = pd.merge(test, train_cols, on=['subject_id','date'])
test = pd.merge(test, train_label, on=['subject_id','date'])
test

,date,month,weekday,weekend,subject_id,activity,IN_VEHICLE,ON_BICYCLE,STILL,UNKNOWN,...,e4Temp_max,pmEmotion_y,pmStress_y,Q1,Q2,Q3,S1,S2,S3,S4
0,2020-08-31,8,1,0,user01,779,101,0,523,127,...,35.07,4.0,3.0,0,0,0,0,1,1,1
1,2020-09-05,9,0,1,user01,698,40,0,530,77,...,35.11,4.0,4.0,1,0,1,1,1,0,1
2,2020-09-07,9,1,0,user01,731,41,0,589,54,...,35.50,4.0,4.0,1,0,1,1,1,1,1
3,2020-09-12,9,0,1,user01,414,15,0,323,59,...,35.25,4.0,4.0,0,0,1,0,0,1,1
4,2020-09-13,9,0,1,user01,658,84,0,489,52,...,35.39,4.0,4.0,0,0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,2020-09-21,9,1,0,user30,836,5,0,762,32,...,35.49,3.0,3.0,0,0,1,0,1,0,0
284,2020-09-22,9,1,0,user30,757,6,0,650,59,...,35.16,3.0,3.0,0,0,1,0,1,1,1
285,2020-09-23,9,1,0,user30,841,9,1,733,54,...,35.61,3.0,3.0,0,0,1,0,1,1,0
286,2020-09-24,9,1,0,user30,755,4,1,663,47,...,35.31,3.0,3.0,1,0,1,0,1,1,1


In [435]:
test.to_csv('../data/train_final_v2.csv')